<a href="https://colab.research.google.com/github/kurtlee1984/Modification_code/blob/main/YOLOv4_Pytorch_modification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 將 Cola b與 google_drive 綁定
# 連接到指定帳號,並複製授權碼貼上

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 當前設備顯卡參數列表明細:

!nvidia-smi

In [1]:
# 從 Github 下載 YOLOv4 資料庫

!git clone https://github.com/roboflow-ai/pytorch-YOLOv4.git

Cloning into 'pytorch-YOLOv4'...
remote: Enumerating objects: 398, done.
remote: Total 398 (delta 0), reused 0 (delta 0), pack-reused 398
Receiving objects: 100% (398/398), 926.53 KiB | 12.19 MiB/s, done.
Resolving deltas: 100% (225/225), done.


In [2]:
# 安裝需求套件

%cd /content/pytorch-YOLOv4
!pip install -r requirements.txt

# import os
# os.kill(os.getpid(), 9)

/content/pytorch-YOLOv4
     |████████████████████████████████| 20.2 MB 5.9 MB/s 
     |████████████████████████████████| 753.4 MB 6.7 kB/s 
     |████████████████████████████████| 12.6 MB 302 kB/s 
     |████████████████████████████████| 59 kB 4.7 MB/s 
     |████████████████████████████████| 124 kB 56.4 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
ERROR: pip's dependency resolver does not currently tak

In [3]:
# 下載Pytorch 的 YOLOv4 權重

%cd /content/pytorch-YOLOv4/cfg/
!gdown https://drive.google.com/uc?id=1fcbR0bWzYfIEdLJPzOsn4R5mlvR6IQyA

/content/pytorch-YOLOv4/cfg
Downloading...
From: https://drive.google.com/uc?id=1fcbR0bWzYfIEdLJPzOsn4R5mlvR6IQyA
To: /content/pytorch-YOLOv4/cfg/yolov4.conv.137.pth
170MB [00:01, 110MB/s]


In [ ]:
# 轉換當前路徑目錄
# 下載資料集,並於 google_drive 取得壓縮檔的 id
# id取得方法,到 google_drive 指定壓縮檔,滑鼠右鍵選擇取得連結,更改權限由限制->知道連結的使用者,複製連結在筆記本上
# https://drive.google.com/file/d/這裡就是壓縮檔的id/view?usp=sharing

%cd /content
!gdown --id '1s8LGb6C2HD7z2ecXiY-njxAMRMBK1Fs-' -O dataset.zip

In [ ]:
# 解壓縮,注意壓縮檔名稱需與代碼一致
# 解壓縮完,於右邊選擇檔案,看有無已解壓縮的資料夾

!unzip -q dataset.zip

In [ ]:
# 複製目錄檔案,並創建一個data資料夾存放

%cp train/_annotations.txt train/train.txt
%cp train/_annotations.txt train.txt

%mkdir data
%cp valid/_annotations.txt data/val.txt
%cp valid/*.jpg train/

In [ ]:
# 從 classes.txt 中定義類別數

def file_len(fname):
  with open(fname) as f:
    for i, l in enumerate(f):
      pass
  return i + 1

num_classes = file_len('train/_classes.txt')

In [ ]:
# 列印並確認類別數

print(num_classes)

In [ ]:
# 以自己所需求的 epoch 數量去訓練 YOLOv4, 可自行修改所需要的epochs數量,這裡是 500 epochs
# 參數說明: -b (batch size), -s (在batch中subdivisions的數量), -l (learning rate), -g ( Gpu設定定, 0為第一顆Gpu), pretrained (是否有預訓練的 weight),
#       -classes (類別數量), -dir (數練資料的路徑位置), -epoch (epoch數量)

!python ./pytorch-YOLOv4/train.py -b 2 -s 1 -l 0.001 -g 0 -pretrained /content/pytorch-YOLOv4/cfg/yolov4.conv.137.pth -classes {num_classes} -dir ./train -epochs 500

In [ ]:
# 使用 tensorboard 可視化套件
# 讀取"runs"資料夾下的結果

%load_ext tensorboard
%tensorboard --logdir 

In [ ]:
# 列出訓練過程中的 checkpoints

!ls checkpoints

In [ ]:
# 隨機拿取 test 資料夾的照片,並顯示出來

import os
test_images = [f for f in os.listdir('test') if f.endswith('.jpg')]
import random
img_path = "test/" + random.choice(test_images);

In [ ]:
# 選擇一個好的 epoch 結果, 用來進行 inference

%%time
!python /content/pytorch-YOLOv4/models.py {num_classes} checkpoints/Yolov4_epoch46.pth {img_path} test/_classes.txt

In [ ]:
# 可視化 inference 的結果

from IPython.display import Image
Image('predictions.jpg')